In [1]:
import os
os.environ["OTEL_SDK_DISABLED"] = "true"

In [2]:
import sys
print(sys.getrecursionlimit())

3000


In [3]:
import resource, sys
resource.setrlimit(resource.RLIMIT_STACK, (2**29,-1))
sys.setrecursionlimit(10**6)

In [ ]:
# crewai_preauth_provider.py
from crewai import Agent, Task, Crew
from crewai import LLM
from crewai.tools import tool
from typing import Any, Dict

# ---------- TOOL SIMULATING INSURER SUBMISSION ----------
@tool
def submit_preauth_to_insurer(preauth_data: dict) -> str:
    """
    Simulates submission of a pre-auth form to the insurer's portal.
    """
    return f"✅ Preauth submitted successfully. Reference ID: {hash(str(preauth_data)) % 100000}"

# ---------- LLM SETUP ----------
llm = LLM(model="ollama/deepseek-r1:1.5b", base_url="http://localhost:11434")

# ---------- AGENTS ----------

front_registration_agent = Agent(
    role="Front Registration Desk Assistant",
    goal="Collect accurate patient demographics, insurance details, and clinical notes to initiate the preauthorization process.",
    backstory=(
        "You are an AI-assisted front desk coordinator in a hospital. "
        "Your job is to gather complete and accurate information from patients, "
        "including demographics, insurance card images, and the attending physician's clinical notes. "
        "You ensure that an Electronic Medical Record (EMR) is created or updated, and associate the patient with the correct physician and department. "
        "Once all required data is verified and structured, you pass it to the diagnosis_billing_agent for coding and cost estimation."
    ),
    verbose=True,
    llm=llm,
    input_keys=["patient_info", "insurance_card", "clinical_notes"]
)

diagnosis_billing_agent = Agent(
    role="AI-Powered Medical Plan Assistant",
    goal="Translate clinical documentation into standardized codes and generate accurate treatment cost estimates.",
    backstory=(
        "You are a clinical NLP expert agent embedded in a hospital's preauthorization pipeline. "
        "You process physician notes to extract key medical concepts such as diagnoses, procedures, and comorbidities. "
        "You then map these concepts to standard coding systems like ICD-10, CPT/HCPCS, and DRG where applicable. "
        "You also estimate procedure costs, length of stay, and post-treatment requirements based on internal hospital pricing databases and historical data. "
        "Your outputs include standardized codes, cost estimates, and a clinical justification summary, which are then forwarded to the Insurance Desk Agent."
    ),
    verbose=True,
    llm=llm,
    input_keys=["clinical_notes", "pricing_data", "coding_reference"]
)

insurance_desk_agent = Agent(
    role="AI-Powered Insurance Coordinator",
    goal="Validate patient insurance eligibility and coverage for the planned diagnosis and procedure.",
    backstory=(
        "You are an expert AI agent embedded in the hospital’s insurance desk workflow. "
        "Your job is to verify the patient’s insurance details using payer APIs, including policy status, plan coverage, and network affiliation. "
        "You cross-check the diagnosis and procedure codes from the Diagnosis Billing Agent against the patient’s plan to ensure coverage, "
        "flag any exclusions, and identify inconsistencies like inactive policies or mismatched patient names. "
        "Your validation ensures only complete and accurate requests move forward for preauthorization submission."
    ),
    verbose=True,
    llm=llm,
    input_keys=["insurance_card", "policy_eligibility", "coding_reference"]
)

preauth_form_generator_agent = Agent(
    role="AI-Powered Preauthorization Compiler and Submitter",
    goal="Assemble, format, and submit a compliant preauthorization request to the insurer.",
    backstory=(
        "You are a final-stage automation agent in the hospital’s preauthorization workflow. "
        "You gather structured and validated data from upstream agents—including patient demographics, insurance details, diagnosis codes, procedure codes, cost estimates, and clinical justification. "
        "You format this data according to payer-specific standards such as EDI 278, FHIR bundles, PDFs, or web form templates. "
        "You also simulate submission to the insurer’s portal, ensuring all data is compliant and complete. "
    ),
    verbose=True,
    llm=llm,
    input_keys=[
        "patient_info", "insurance_card", "pricing_data",
        "coding_reference", "insurer_submission"
    ]
)

# ---------- TASKS ----------

collect_patient_info = Task(
    description="Collect patient demographics such as first name, last name, dob, gender, and contact details from the input. \
       Input: {patient_info} ",
    expected_output="Structured patient demographic data in EMR format.",
    agent=front_registration_agent
)

scan_insurance_card = Task(
    description="Scan and digitize both sides of the patient's insurance card. Extract policy number, insurer name, and group ID from the input. \
       Input: {insurance_card} ",
    expected_output="JSON with insurance card details attached to patient's EMR.",
    agent=front_registration_agent
)

gather_clinical_notes = Task(
    description="Receive and store initial physician documentation, including presenting symptoms, working diagnosis, and treatment recommendation. \
        Input Clinical notes: {clinical_notes} ",
    expected_output="Structured clinical note summary linked to the patient’s record.",
    agent=front_registration_agent
)

extract_clinical_concepts = Task(
    description="Extract key clinical concepts such as diagnoses, procedures, and comorbidities. Use NLP to extract diagnosis, procedure, and comorbidities from physician notes. \
        Input: {clinical_notes} ",
    expected_output="A list of clinical concepts extracted.",
    agent=diagnosis_billing_agent
)

generate_standard_codes = Task(
    description="Map clinical concepts to ICD-10, CPT/HCPCS, and DRG codes using the standard coding rules. \
        Coding reference: {coding_reference} ",
    expected_output="A Mapping of codes for diagnosis and procedures.",
    agent=diagnosis_billing_agent
)

estimate_treatment_costs = Task(
    description="Estimate total treatment cost based on procedure, length of stay, and consumables using hospital pricing data. \
        Reference treatment cost data: {pricing_data} ",
    expected_output="Itemized cost estimation in structured format.",
    agent=diagnosis_billing_agent
)

summarize_clinical_justification = Task(
    description="Generate a guideline-compliant clinical justification summary for the planned treatment. \
        Clinical notes: {clinical_notes} ",
    expected_output="Clear justification aligned with MCG or InterQual criteria.",
    agent=diagnosis_billing_agent
)

merge_patient_and_treatment_data = Task(
    description="Merge data from prior agents including patient info, insurance, diagnosis/procedure codes, cost estimate, and justification. \
        ",
    expected_output="Preauthorization data packet ready for formatting.",
    agent=preauth_form_generator_agent
)

format_preauth_form = Task(
    description="Format the compiled data into insurer-compliant templates (EDI 278, FHIR bundle, PDF, or web form). \
        ",
    expected_output="Well-structured and compliant preauth request document.",
    input_keys=["insurer_submission"],
    agent=preauth_form_generator_agent
)

# ---------- CREW SETUP ----------

provider_preauth_crew = Crew(
    agents=[
        front_registration_agent,
        diagnosis_billing_agent,
        insurance_desk_agent,
        preauth_form_generator_agent
    ],
    tasks=[
        collect_patient_info,
        scan_insurance_card,
        gather_clinical_notes,
        extract_clinical_concepts,
        generate_standard_codes,
        estimate_treatment_costs,
        summarize_clinical_justification,
        merge_patient_and_treatment_data,
        format_preauth_form,
    ],
    model="ollama/mistral:latest",
    verbose=True,
    process_name="Hospital PreAuthorization Provider-Side Automation"
)

# ---------- RUN WORKFLOW ----------

if __name__ == "__main__":
    input_data: Dict[str, Any] = {
        "patient_info": {
            "first_name": "Rajesh",
            "last_name": "Kumar",
            "dob": "1980-05-12",
            "gender": "Male",
            "contact": {
                "phone": "+91-9876543210",
                "email": "rajesh.kumar@example.com"
            },
            "address": "Hyderabad, Telangana, India",
            "ehr_id": "EHR-998877",
            "physician": {
                "name": "Dr. Anjali Mehta",
                "department": "Orthopedics",
                "npi": "1234567890"
            }
        },
        "insurance_card": {
            "insurer_name": "Star Health Insurance",
            "policy_number": "STAR-HI-20394857",
            "group_number": "GRP-112233",
            "valid_from": "2024-01-01",
            "valid_to": "2025-12-31",
            "network_affiliation": "In-Network"
        },
        "clinical_notes": {
            "symptoms": "Severe pain and stiffness in left knee",
            "preliminary_diagnosis": "Left Knee Osteoarthritis",
            "recommended_procedure": "Arthroscopic Knee Surgery",
            "risk_factors": ["Hypertension", "Obesity"],
            "notes_text": """
                The patient presents with chronic left knee pain for over 6 months, worsening with movement. 
                X-ray confirms osteoarthritis. Recommended arthroscopic intervention with pre/post rehab. 
                No known drug allergies. Pre-op evaluation needed.
            """
        },
        "pricing_data": {
            "procedure_name": "Arthroscopic Knee Surgery",
            "procedure_code": "29881",
            "procedure_cost": 48000,
            "estimated_length_of_stay": "2 days",
            "pre_post_care_cost": 7500,
            "consumables": 3200,
            "total_estimate": 58700
        },
        "coding_reference": {
            "icd_codes": {
                "M17.12": {
                    "description": "Unilateral primary osteoarthritis, left knee",
                    "category": "Disorder",
                    "related_symptoms": [
                        "knee pain",
                        "joint stiffness",
                        "reduced mobility"
                    ],
                    "associated_procedures": [
                        "Arthroscopy",
                        "Knee replacement"
                    ]
                }
            },
            "cpt_codes": {
                "29881": {
                    "description": "Arthroscopy, knee, surgical; with meniscectomy (medial OR lateral including any meniscal shaving)",
                    "category": "Procedure",
                    "used_for": [
                        "Meniscal tear",
                        "Osteoarthritis",
                        "Knee pain"
                    ],
                    "typical_duration": "1 hour"
                }
            },
            "drg_code": {
                "470": {
                    "description": "Major joint replacement or reattachment of lower extremity",
                    "category": "Procedure Grouping",
                    "covered_procedures": [
                        "Total Knee Replacement",
                        "Hip Replacement"
                    ],
                    "hospital_stay_avg": "3-4 days"
                }
            }
        }
    }

    result = provider_preauth_crew.kickoff(inputs=input_data)
    print("\n=== Final Output ===")
    print(result)


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 89366bfe-0ec5-45af-9937-5ef4b7af383d                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Front Registration Desk Assistant                                                                       │
│                                                                                                                 │
│  Task: Collect patient demographics such as first name, last name, dob, gender, and contact details from the    │
│  input.        Input: {'first_name': 'Rajesh', 'last_name': 'Kumar', 'dob': '1980-05-12', 'gender': 'Male',     │
│  'contact': {'phone': '+91-9876543210', 'email': 'rajesh.kumar@example.com'}, 'address': 'Hyderabad,            │
│  Telangana, India', 'ehr_id': 'EHR-998877', 'physician': {'name': 'Dr. Anjali Mehta', 'department':             │
│  'Orthopedics', 'npi': '1234567890'}}                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Front Registration Desk Assistant                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  <think>                                                                                                        │
│  Okay, I need to help Rajesh with gathering his patient demographics for front registration. Let me go through  │
│  each piece of information step by step.                                                                        │
│                                                                                                                 │
│  First, the name seems straightforward: Rajesh Kumar. Now, for the dob, it's given as 1980-05-12, which is 19   │
│  years and 6 months old when he was born. Since he's a patient, I should check if the department requires him   │
│  to be at least 18. If yes, that might affect some info entries.                                                │
│                                                                                                                 │
│  Gender is specified as Male, so no issue there. Contact details are provided in both phone and email formats.  │
│  The phone number is +91-9876543210. I think sometimes people prefer landline numbers, but the given one seems  │
│  correct based on EHR standards. Email address should be valid for Rajesh.                                      │
│                                                                                                                 │
│  Address is given as Hyderabad, Telangana, India. That's a standard city in India, so that makes sense. EHR ID  │
│  is provided as EHR-998877, which I assume is specific to his department or center.                             │
│                                                                                                                 │
│  Physician information includes Dr. Anjali Mehta from Orthopedics with NPI 1234567890. That’s straightforward   │
│  and probably used for coding later.                                                                            │
│                                                                                                                 │
│  Putting it all together in EMR format, Rajesh should be listed under the EHR ID and department. The first      │
│  name is first, followed by last name and dob as a date. Gender comes next, then contact details separated      │
│  into phone and email. Address follows, and physician information is at the bottom with their NPI.              │
│                                                                                                                 │
│  I think that's all he needs. I just need to make sure each field is accurately represented in EMR format       │
│  without any errors.                                                                                            │
│  </think>                                                                                                       │
│                                                                                                                 │
│  EHR-998877 | Rajesh Kumar | 1980-05-12 | Male | +91-9876543210 | rajesh.kumar@example.com | Hyderabad,         │
│  Telangana, India | Orthopedics | NPI: 1234567890                                                               │
│                                                                                                                 │
╰────────────────────────────────────────────────────────

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: aa2b3d80-5c95-4308-933a-fd9180be1958                                                                     │
│  Agent: Front Registration Desk Assistant                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────────── LLM Error ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  ❌ LLM Call Failed                                                                                             │
│  Error: litellm.APIConnectionError: OllamaException - litellm.Timeout: Connection timed out after 600.0         │
│  seconds.                                                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


 An unknown error occurred. Please check the details below.



╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: 25058dad-0d6e-4cc8-8957-e42473fc70c8                                                                     │
│  Agent: AI-Powered Medical Plan Assistant                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: 89366bfe-0ec5-45af-9937-5ef4b7af383d                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output:                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

APIConnectionError: litellm.APIConnectionError: OllamaException - litellm.Timeout: Connection timed out after 600.0 seconds.